In [65]:
import numpy as np
import scipy as sc
from scipy import sparse
from scipy.sparse import csgraph
from scipy.spatial import distance

x = 5
y = 5
pocatek = (0,0)
matrixDensity = 0.1

n = int(x * y * matrixDensity)
space = np.zeros((x*y), dtype=int)
rngX = np.random.choice(range(x*y), n, replace=False)
for i in rngX:
    space[i] = 1

points = [pocatek]
distanceMatrix = np.zeros(1, dtype=int)
space = space.reshape(x, y)
display(space)
for i in range(len(space)):
    for j in range(len(space[i])):
        if space[i][j] == 1:
            distanceMatrix_t = np.zeros((len(distanceMatrix)+1,len(distanceMatrix)+1), dtype=int)       
            distanceMatrix_t[:-1, :-1] = distanceMatrix
            distanceMatrix = distanceMatrix_t
            pointNumber = len(points)
            for point in range(len(points)):
                distanceMatrix[pointNumber][point] = distance.cityblock(points[point], (i, j))
                distanceMatrix[point][pointNumber] = distance.cityblock(points[point], (i, j))
            points.append((i, j))

display(distanceMatrix)


array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

array([[0, 4, 3],
       [4, 0, 7],
       [3, 7, 0]])